In [1]:
import autohf as hf
import autograd.numpy as anp
import autograd
import numpy as np
from autograd.differential_operators import make_jvp
import chemistry as chem
np.set_printoptions(linewidth=300)
from pennylane import qchem

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
# We define some basic information about the molecule
molecule = chem.BeH2()
R = anp.array([0.0, 1.0, 0.5, 0.0, -2.4359718264299817, 0.0, 0.0, 2.4359718264299817, 0.0]) # Optimal geometry
structure = ['Be', 'H', 'H']

charges = [4, 1, 1]
num_elecs = 6

R1, R2, R3 = anp.array(R[0:3]), anp.array(R[3:6]), anp.array(R[6:9]) # Atomic coordiantes

In [3]:
# We also define the basis set
A1, A2, A3 = hf.basis_set_params("sto-3g", structure) # Gets default basis set parameters

a1 = []
for func in A1:
    L, exp, coeff = func
    a1.append(hf.AtomicBasisFunction(L, C=anp.array(coeff), A=anp.array(exp)))
    
a2 = []
for func in A2:
    L, exp, coeff = func
    a2.append(hf.AtomicBasisFunction(L, C=anp.array(coeff), A=anp.array(exp)))

a3 = []
for func in A3:
    L, exp, coeff = func
    a3.append(hf.AtomicBasisFunction(L, C=anp.array(coeff), A=anp.array(exp)))
    
basis_set = [a1, a2, a3]

In [4]:
num_act_orb = 6
num_act_elec = 4
wires = list(range(12))
core, active = qchem.active_space(6, 7, active_electrons=num_act_elec, active_orbitals=num_act_orb) # Prepares active space

In [5]:
h_fn = chem.hamiltonian(molecule, basis_set, num_elecs, charges, wires, core=core, active=active)

In [6]:
vec = np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
dh_fn = chem.d_hamiltonian(molecule, basis_set, num_elecs, charges, wires, core=core, active=active)

In [12]:
def fd(func, vec, delta=0.002):
    return lambda R : (1 / delta) * (func(R + (delta/2) * vec) - func(R - (delta/2) * vec))

In [13]:
h_fd = fd(h_fn, vec)(R)
#h_fd_original = fd(molecule.hamiltonian(active_electrons=4, active_orbitals=6), vec)(R)

In [14]:
dh = dh_fn(R, vec)

In [15]:
print(list(dh - h_fd))

[-3.672075153850356e-17, 1.3007745610014145e-17, -718.9505015941421, -1.520073129868127e-14, 1.3345285491612619e-15, -1.0591531296238568e-16, -3.335244832670057e-17, -718.9505015941421, 6.702810450497356e-15, -1.4594179321332372e-13, -1907.6802688812818, 1222.1513169024588, 2575.0276704255043, -1.5414454651932474e-14, -1.4552982308086756e-13, -1.4165905147521446e-15, 3.946495907847236e-14, -2.4549156110720283e-14, 5.4312457309357853e-14, 1.3345285491612585e-15, -1907.6802688812822, 3.845101320676392e-14, -4.721644512734787e-15, 2.6108355654819917e-15, -4.154495226979289e-15, -1.0591531296238574e-16, 1222.151316902459, -2.4703654920299467e-14, 2.6108355654819917e-15, -2.532672334066564e-16, 3.736022603564289e-16, -3.335244832670053e-17, 2575.0276704255034, 5.4550114425566676e-14, -4.154495226979286e-15, 3.73602260356429e-16, 1.1866094288639304e-17, 1.1852112407000355e-18, -0.5435032497024479, 1.214306433183765e-17, 2.699356556250297e-18, 3.1088201533348995e-18, 5.652912577295188e-18, -0

In [12]:
print(list(h_fd))

[0.0, 0.0, 0.0, -0.08203334601389076, 0.0, 0.0, 0.0, 0.0, 0.0, 2.2201397469190476e-12, 0.0, 0.0, 0.0, -0.08203334601389123, 2.220120941614056e-12, 0.0, 3.9415874844576135e-13, 0.06565069424901271, 1.2208559222241876e-11, 0.0, 0.0, 3.9415943521829956e-13, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06565069424901203, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2208667410745156e-11, 0.0, 0.0, 0.0, 0.0, 0.0, 0.043451711608894626, 0.0, 0.0, 0.0, 0.0, 0.0, -4.099639464710814e-15, 0.0, 0.0, 0.0, 0.043451711608894626, -4.108109794183357e-15, 0.0, -4.079191677116637e-14, 0.008123008885361507, -5.326736095398119e-13, 0.0, 0.0, -4.0791904323090827e-14, 0.0, 0.0, 0.0, 0.0, 0.0, 0.008123008885361292, 0.0, 0.0, 0.0, 0.0, 0.0, -5.327244315166471e-13, 0.0, 0.0, 0.0, 0.0, 0.0, -4.097521882342678e-15, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03974045911002244, 0.0, 0.0, 0.0, -8.392042452393025e-13, 0.040898946459552984, 0.0, 8.14491766976777e-18, 1.4712962293807197e-13, -0.04173788303341822, 0.0, 0.0, 1.0341225017144277e-17, 0.0, 0.0, 0.0, 0.0, 0.0,